In [1]:
#importing dependancies
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [2]:
#import raw data for discovery and cleaning using pandas
df = pd.read_csv(r'raw_data\spreadspoke_scores.csv')

In [3]:
#display df

df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN


# This Journal will go over betting odds


In [4]:
# Checking how many rows don't have gambling data or scores, removing those rows

df.isnull().sum()

schedule_date              0
schedule_season            0
schedule_week              0
schedule_playoff           0
team_home                  0
score_home               272
score_away               272
team_away                  0
team_favorite_id        2751
spread_favorite         2751
over_under_line         2761
stadium                    0
stadium_neutral            0
weather_temperature     1548
weather_wind_mph        1564
weather_humidity        5597
weather_detail         10946
dtype: int64

In [5]:
df = df[['schedule_date','schedule_season', 'schedule_week','team_home','score_home','team_away','score_away',
        'team_favorite_id','spread_favorite','over_under_line']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14073 entries, 0 to 14072
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   schedule_date     14073 non-null  object 
 1   schedule_season   14073 non-null  int64  
 2   schedule_week     14073 non-null  object 
 3   team_home         14073 non-null  object 
 4   score_home        13801 non-null  float64
 5   team_away         14073 non-null  object 
 6   score_away        13801 non-null  float64
 7   team_favorite_id  11322 non-null  object 
 8   spread_favorite   11322 non-null  float64
 9   over_under_line   11312 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 1.1+ MB


In [6]:
# Dropping null values 

df = df.dropna()
df.head()



,schedule_date,schedule_season,schedule_week,team_home,score_home,team_away,score_away,team_favorite_id,spread_favorite,over_under_line
350,1/14/1968,1967,Superbowl,Green Bay Packers,33.0,Oakland Raiders,14.0,GB,-13.5,43
538,1/12/1969,1968,Superbowl,Baltimore Colts,7.0,New York Jets,16.0,IND,-18.0,40
727,1/11/1970,1969,Superbowl,Kansas City Chiefs,23.0,Minnesota Vikings,7.0,MIN,-12.0,39
916,1/17/1971,1970,Superbowl,Baltimore Colts,16.0,Dallas Cowboys,13.0,IND,-2.5,36
1105,1/16/1972,1971,Superbowl,Dallas Cowboys,24.0,Miami Dolphins,3.0,DAL,-6.0,34


In [7]:
df['points_scored'] = df['score_home'] + df['score_away']
df

,schedule_date,schedule_season,schedule_week,team_home,score_home,team_away,score_away,team_favorite_id,spread_favorite,over_under_line,points_scored
350,1/14/1968,1967,Superbowl,Green Bay Packers,33.0,Oakland Raiders,14.0,GB,-13.5,43,47.0
538,1/12/1969,1968,Superbowl,Baltimore Colts,7.0,New York Jets,16.0,IND,-18.0,40,23.0
727,1/11/1970,1969,Superbowl,Kansas City Chiefs,23.0,Minnesota Vikings,7.0,MIN,-12.0,39,30.0
916,1/17/1971,1970,Superbowl,Baltimore Colts,16.0,Dallas Cowboys,13.0,IND,-2.5,36,29.0
1105,1/16/1972,1971,Superbowl,Dallas Cowboys,24.0,Miami Dolphins,3.0,DAL,-6.0,34,27.0
...,...,...,...,...,...,...,...,...,...,...,...
13796,1/21/2024,2023,Division,Buffalo Bills,24.0,Kansas City Chiefs,27.0,BUF,-2.5,46,51.0
13797,1/21/2024,2023,Division,Detroit Lions,31.0,Tampa Bay Buccaneers,23.0,DET,-6.0,49.5,54.0
13798,1/28/2024,2023,Conference,Baltimore Ravens,10.0,Kansas City Chiefs,17.0,BAL,-4.5,44,27.0
13799,1/28/2024,2023,Conference,San Francisco 49ers,34.0,Detroit Lions,31.0,SF,-7.5,53.5,65.0


In [8]:
'''
Removing white space from created column then changing data type to float for further evaluation

'''

df['over_under_line'] = df['over_under_line'].str.strip()

df['over_under_line'] = pd.to_numeric(df['over_under_line'])


In [9]:
df = df.dropna()

In [10]:
df[['over_under_line', 'score_home','score_away']] = df[['over_under_line', 'score_home','score_away']].astype(int)

c:\Users\palum\anaconda3\envs\GreatRes\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [11]:
df.dtypes


schedule_date        object
schedule_season       int64
schedule_week        object
team_home            object
score_home            int32
team_away            object
score_away            int32
team_favorite_id     object
spread_favorite     float64
over_under_line       int32
points_scored       float64
dtype: object

In [12]:
''' creating over/under column to test results of the game that was played'''
conditions = [
    (df['points_scored'] > df['over_under_line']),
    (df['points_scored'] < df['over_under_line']),
     (df['points_scored'] == df['over_under_line'])
]

results = ['over','under','push']
df['over_under_result'] = np.select(conditions,results)

c:\Users\palum\anaconda3\envs\GreatRes\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [13]:
df.head()

,schedule_date,schedule_season,schedule_week,team_home,score_home,team_away,score_away,team_favorite_id,spread_favorite,over_under_line,points_scored,over_under_result
350,1/14/1968,1967,Superbowl,Green Bay Packers,33,Oakland Raiders,14,GB,-13.5,43,47.0,over
538,1/12/1969,1968,Superbowl,Baltimore Colts,7,New York Jets,16,IND,-18.0,40,23.0,under
727,1/11/1970,1969,Superbowl,Kansas City Chiefs,23,Minnesota Vikings,7,MIN,-12.0,39,30.0,under
916,1/17/1971,1970,Superbowl,Baltimore Colts,16,Dallas Cowboys,13,IND,-2.5,36,29.0,under
1105,1/16/1972,1971,Superbowl,Dallas Cowboys,24,Miami Dolphins,3,DAL,-6.0,34,27.0,under


In [14]:
df['team_favorite_id'].unique() 



array(['GB', 'IND', 'MIN', 'DAL', 'MIA', 'PIT', 'LVR', 'TB', 'CHI', 'DEN',
       'KC', 'LAR', 'NO', 'NYJ', 'PHI', 'SEA', 'TEN', 'PICK', 'CIN', 'NE',
       'ARI', 'LAC', 'CLE', 'WAS', 'BUF', 'ATL', 'DET', 'NYG', 'SF',
       'BAL', 'JAX', 'CAR', 'HOU', 'LV'], dtype=object)

In [15]:
df['team_home'].unique()


array(['Green Bay Packers', 'Baltimore Colts', 'Kansas City Chiefs',
       'Dallas Cowboys', 'Miami Dolphins', 'Minnesota Vikings',
       'Tampa Bay Buccaneers', 'Buffalo Bills', 'Chicago Bears',
       'Denver Broncos', 'Los Angeles Rams', 'New Orleans Saints',
       'New York Jets', 'Philadelphia Eagles', 'Seattle Seahawks',
       'St. Louis Cardinals', 'Washington Redskins',
       'New England Patriots', 'Detroit Lions', 'New York Giants',
       'Pittsburgh Steelers', 'San Diego Chargers', 'San Francisco 49ers',
       'Atlanta Falcons', 'Cincinnati Bengals', 'Cleveland Browns',
       'Houston Oilers', 'Oakland Raiders', 'Los Angeles Raiders',
       'Indianapolis Colts', 'Phoenix Cardinals', 'Arizona Cardinals',
       'Jacksonville Jaguars', 'St. Louis Rams', 'Carolina Panthers',
       'Baltimore Ravens', 'Tennessee Oilers', 'Tennessee Titans',
       'Houston Texans', 'Los Angeles Chargers',
       'Washington Football Team', 'Las Vegas Raiders',
       'Washington Comman

In [16]:
replacements = {'GB':'Green Bay Packers', 'IND':'Baltimore Colts', 'MIN':'Minnesota Vikings', 'DAL':'Dallas Cowboys', 'MIA':'Miami Dolphins', 
                'PIT':'Pittsburgh Steelers', 'LVR':'Las Vegas Raiders', 'TB':'Tampa Bay Buccaneers', 'CHI':'Chicago Bears', 'DEN':'Denver Broncos',
                 'KC': 'Kansas City Chiegs', 'LAR':'Los Angeles Rams', 'NO': 'New Orleans Saints', 'NYJ': 'New York Jets', 'PHI':'Philadelphia Eagles', 'SEA':'Seattle Seahawks',
                'TEN':'Tennessee Titans', 'PICK':'PICK', 'CIN':'Cincinnati Bengals', 'NE':'New England Patriots',
                'ARI':'Arizona Cardinals', 'LAC': 'Los Angeles Chargers', 'CLE':'Cleveland Browns', 'WAS':'Washington Commanders', 'BUF':'Buffalo Bills',
                'ATL':'Atlanta Falcons', 'DET':'Detroit Lions', 'NYG':'New York Giants', 'SF':'San Francisco 49ers',
                'BAL':'Baltimore Ravens', 'JAX':'Jacksonville Jaguars', 'CAR':'Carolina Panthers', 'HOU':'Houston Texans', 'LV':'Las Vegas Raiders', 'nan':'nan'}

In [17]:
df['team_favorite_id'] = df['team_favorite_id'].replace(replacements)

c:\Users\palum\anaconda3\envs\GreatRes\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
df.head()

,schedule_date,schedule_season,schedule_week,team_home,score_home,team_away,score_away,team_favorite_id,spread_favorite,over_under_line,points_scored,over_under_result
350,1/14/1968,1967,Superbowl,Green Bay Packers,33,Oakland Raiders,14,Green Bay Packers,-13.5,43,47.0,over
538,1/12/1969,1968,Superbowl,Baltimore Colts,7,New York Jets,16,Baltimore Colts,-18.0,40,23.0,under
727,1/11/1970,1969,Superbowl,Kansas City Chiefs,23,Minnesota Vikings,7,Minnesota Vikings,-12.0,39,30.0,under
916,1/17/1971,1970,Superbowl,Baltimore Colts,16,Dallas Cowboys,13,Baltimore Colts,-2.5,36,29.0,under
1105,1/16/1972,1971,Superbowl,Dallas Cowboys,24,Miami Dolphins,3,Dallas Cowboys,-6.0,34,27.0,under
